In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [2]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [3]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [4]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [5]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('object_detection/test_images/image1.jpg'),
 WindowsPath('object_detection/test_images/image2.jpg')]

In [6]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [8]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  #image_np = np.array(Image.open(image_path))
  image_np=image_path
    # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  #print(output_dict)
  #print(category_index)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2,min_score_thresh=0.5)
  #display(Image.fromarray(image_np))
  person_detection(output_dict['detection_classes'],output_dict['detection_scores'],image_np)
  cv2.imshow('LIVE',image_np)

In [9]:
def person_detection(classes,scores,image_np):
    
    if (classes[0]==1 and scores[0]>0.8):
        print('PERSON DETECTED!')
        cv2.imwrite('detected.jpg',image_np)

In [13]:
import cv2

#video=cv2.VideoCapture('C://Users/User/Desktop/DAY06/Chicago traffic and pedestrians at a crosswalk.mp4')
video=cv2.VideoCapture(0)

while(True):
    ret,img=video.read()
    #img=cv2.resize(img,(320,180))
    show_inference(detection_model,img)
    cv2.waitKey(1)

PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!
PERSON DETECTED!


KeyboardInterrupt: 